In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
a=test['PassengerId']

In [ ]:
train

In [ ]:
train.drop(columns=[ "PassengerId","Name", "Ticket", "Cabin"], inplace=True)
test.drop(columns=[ "PassengerId","Name", "Ticket", "Cabin"], inplace=True)

In [ ]:
train

In [ ]:
test

In [ ]:
train['Embarked'].mode()
train.groupby('Embarked').sum()

In [ ]:
train['Embarked'].fillna('S', inplace=True)
test['Embarked'].fillna('S', inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le_sex = LabelEncoder()
train['Sex'] = le_sex.fit_transform(train['Sex'])
unique_op_sex = train['Sex'].unique()
print("Label Encoder Parameters Sex", le_sex.inverse_transform(unique_op_sex))

le_embarked = LabelEncoder()
train['Embarked'] = le_embarked.fit_transform(train['Embarked'])
unique_op_embarked = train['Embarked'].unique()
print("Label Encoder Parameters Embarked", le_embarked.inverse_transform(unique_op_embarked))

test['Sex'] = le_sex.fit_transform(test['Sex'])
unique_op_sex = test['Sex'].unique()
print("Label Encoder Parameters Sex", le_sex.inverse_transform(unique_op_sex))

le_embarked = LabelEncoder()
test['Embarked'] = le_embarked.fit_transform(test['Embarked'])
unique_op_embarked = train['Embarked'].unique()
print("Label Encoder Parameters Embarked", le_embarked.inverse_transform(unique_op_embarked))

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# le_sex = LabelEncoder()
# test['Sex'] = le_sex.fit_transform(test['Sex'])
# unique_op_sex = test['Sex'].unique()
# print("Label Encoder Parameters Sex", le_sex.inverse_transform(unique_op_sex))

# le_embarked = LabelEncoder()
# test['Embarked'] = le_embarked.fit_transform(test['Embarked'])
# unique_op_embarked = train['Embarked'].unique()
# print("Label Encoder Parameters Embarked", le_embarked.inverse_transform(unique_op_embarked))

In [ ]:
train

In [ ]:
test


In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(data=train[['Fare']])
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(data=train[['Embarked']])
plt.show()

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3)
train.loc[:, 'Pclass':] = imputer.fit_transform(train.loc[:, 'Pclass':])
test.loc[:, 'Pclass':] = imputer.transform(test.loc[:, 'Pclass':])

In [ ]:
X=train[[ 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y=train[['Survived']]
X_test=test[[ 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
model=DecisionTreeClassifier()
param_grid = {
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid = GridSearchCV(estimator=model, param_grid=param_grid,scoring='accuracy', cv=5, n_jobs=-1, verbose=1)
grid.fit(X, y)
y_pred = grid.predict(X)

y_test=grid.predict(X_test)
accuracy_score(y, y_pred)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Initialize the Decision Tree model
model = DecisionTreeClassifier()

# Define the parameter grid for GridSearchCV
param_grid = {
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create and run the GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=10, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Make predictions
y_pred = grid_search.predict(X)
y_test_dt = grid_search.predict(X_test)

# Evaluate the model performance
accuracy = accuracy_score(y, y_pred)
test_accuracy = accuracy_score(y_test, y_test_dt)

# Output the results
print("Train Accuracy: ", accuracy)
print("Test Accuracy: ", test_accuracy)
print("Best Parameters: ", grid_search.best_params_)


In [ ]:
import pandas as pd
test = pd.read_csv('test.csv')
a = test['PassengerId']
submission = pd.DataFrame({
    'PassengerId': a,
    'Survived': y_test_dt
})
submission.to_csv('submission_dt.csv', index=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
import numpy as np

# Define parameter distribution
param_dist = {
    'n_estimators': [10, 25, 50, 100],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [None, 3, 6, 9],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_leaf_nodes': [None, 3, 6],
}

# Initialize the model
model = RandomForestClassifier(random_state=42)

# Initialize RandomizedSearchCV
rand_search = RandomizedSearchCV(estimator=model,
                                  param_distributions=param_dist,
                                  n_iter=50,             # Number of combinations to try
                                  scoring='accuracy',    # Use accuracy as the scoring metric
                                  cv=5,                  # Use 5-fold cross-validation
                                  n_jobs=-1,            # Use all available cores
                                  verbose=1,            # Verbose output
                                  random_state=42)      # For reproducibility

# Fit the model
rand_search.fit(X, y)

# Predict on the training set
y_pred = rand_search.predict(X)

# Predict on the test set
y_test_rf = rand_search.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(y, y_pred)
test_accuracy = accuracy_score(y_test, y_test_rf)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print("Best Parameters:", rand_search.best_params_)


In [ ]:
import pandas as pd
test = pd.read_csv('test.csv')
a = test['PassengerId']
submission = pd.DataFrame({
    'PassengerId': a,
    'Survived': y_test_rf
})
submission.to_csv('submission_rf.csv', index=False)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Initialize the Gradient Boosting model
model = GradientBoostingClassifier()

# Define an expanded parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150],       # Increased number of estimators
    'learning_rate': [0.01, 0.1, 0.2],    # More varied learning rates
    'max_depth': [3, 4, 5, 6],            # More options for max depth
    'min_samples_split': [2, 5, 10],      # Control overfitting
    'min_samples_leaf': [1, 2, 4],        # Control overfitting
    'subsample': [0.8, 0.9, 1.0]          # Use of subsampling to reduce variance
}

# Create and run the GridSearchCV
grid_search = GridSearchCV(estimator=model,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=5,
                           n_jobs=-1,       # Use all available cores
                           verbose=1)

# Fit the model
grid_search.fit(X, y)

# Make predictions
y_pred = grid_search.predict(X)
y_test_gb = grid_search.predict(X_test)

# Evaluate the model performance
train_accuracy = accuracy_score(y, y_pred)
test_accuracy = accuracy_score(y_test, y_test_gb)

# Output the results
print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Best Parameters: {grid_search.best_params_}")


In [ ]:
import pandas as pd
test = pd.read_csv('test.csv')
a = test['PassengerId']
submission = pd.DataFrame({
    'PassengerId': a,
    'Survived': y_test_gb
})
submission.to_csv('submission_gb.csv', index=False)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Initialize the XGBoost model with some default parameters
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    nthread=1, verbosity=0, random_state=42)

# Define the parameter grid for GridSearchCV
param_grid = {
    'min_child_weight': [1, 5, 10],
    'gamma': [0.5, 1, 1.5, 2, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'max_depth': [3, 4, 5]
}

# Create and run the GridSearchCV
grid_search = GridSearchCV(estimator=xgb,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=5,
                           n_jobs=-1,
                           verbose=1)
grid_search.fit(X, y)

# Make predictions
y_pred = grid_search.predict(X)
y_test_xg = grid_search.predict(X_test)

# Evaluate the model performance
train_accuracy = accuracy_score(y, y_pred)
test_accuracy = accuracy_score(y_test, y_test_xg)

# Output the results
print("Training Accuracy: ", train_accuracy)
print("Test Accuracy: ", test_accuracy)
print("Best Parameters: ", grid_search.best_params_)


In [ ]:
import pandas as pd
test = pd.read_csv('test.csv')
a = test['PassengerId']
submission = pd.DataFrame({
    'PassengerId': a,
    'Survived': y_test_xg
})
submission.to_csv('submission_xg.csv', index=False)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.5, 1.0]
}

# Initialize the AdaBoost model
model = AdaBoostClassifier()

# Create and run the GridSearchCV
grid_search = GridSearchCV(estimator=model,
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1,
                           verbose=1)
grid_search.fit(X, y)

# Make predictions
y_pred = grid_search.predict(X)
y_test_ab = grid_search.predict(X_test)

# Evaluate the model performance
train_accuracy = accuracy_score(y, y_pred)
test_accuracy = accuracy_score(y_test, y_test_ab)

# Output the results
print("Training Accuracy: ", train_accuracy)
print("Test Accuracy: ", test_accuracy)
print("Best Parameters: ", grid_search.best_params_)


In [ ]:
import pandas as pd
test = pd.read_csv('test.csv')
a = test['PassengerId']
submission = pd.DataFrame({
    'PassengerId': a,
    'Survived': y_test_ab
})
submission.to_csv('submission_ab.csv', index=False)